# 3. Inventario
## Importación de librerías

In [13]:
import pandas as pd
import numpy as np

## Declaración de constantes y condiciones

In [14]:
cost_per_storaged_unit = 30 # (Weekly)
demand_avg = 120 # (Weekly)
demand_std = 15 # (Weekly)
# If demand > stock - Then those sells get lost
production_rates = [110, 120, 130] # (Weekly)
production_change_cost = 3000
# If stock < 30 - Produce 130 next week
# If stock > 80 - Produce 110 next week
# Else - Produce 120

## Declaración de función de generación aleatoria de muestras

In [15]:
def simulate_normal_variable(mean: float, std_dev: float, size: int) -> np.ndarray:
    """
    Simulates random variables from a normal (Gaussian) distribution.

    Uses np.random.normal to generate the samples.

    Args:
        mean: The desired mean (average, mu) of the distribution.
        std_dev: The desired standard deviation (sigma) of the distribution.
                 Must be non-negative.
        size: The number of random samples to generate (e.g., number of rows).
              Must be a positive integer.

    Returns:
        A NumPy array of the specified size containing random numbers
        drawn from the normal distribution with the given mean and
        standard deviation.

    Raises:
        ValueError: If std_dev is negative or size is not positive.
    """
    if std_dev < 0:
        raise ValueError("Standard deviation cannot be negative.")
    if not isinstance(size, int) or size <= 0:
         raise ValueError("Size must be a positive integer.")

    # Use np.random.normal to generate the random numbers
    # loc specifies the mean
    # scale specifies the standard deviation
    # size specifies the output shape (number of samples)
    random_samples = np.random.normal(loc=mean, scale=std_dev, size=size)

    return random_samples

## Simulación y registro

In [16]:
# Simulation parameters
week_amount = 52
simulation_amount = 500
u_values = range(30, 81, 10)
initial_state = {
    'Stock': 60,
    'Production rate': 120,
}
# Calculation
iterations = week_amount * simulation_amount * len(u_values)

stock = []
cost = []
weekly_production_rate = []
demand = simulate_normal_variable(demand_avg, demand_std, iterations).astype(int)
upper_bound = np.repeat(u_values, week_amount * simulation_amount)
for i in range(iterations):
    
    if i % week_amount == 0:
        # Reset the state for each simulation
        stock.append(initial_state['Stock']) # We have this in storage at the beginning of the week
        weekly_production_rate.append(initial_state['Production rate']) # Default to 120, this week this will be produced
    else:
        stock_start_of_week = stock[i - 1] # Stock of the end of the previous week.
        # Here we evaluate the production rate given the stock of the start of the week.
        if stock_start_of_week < 30:
            pr = production_rates[2]
        elif stock_start_of_week > upper_bound[i]:
            pr = production_rates[0]
        else:
            pr = weekly_production_rate[i - 1]

        weekly_production_rate.append(pr)
        stock_end_of_week = max(pr + stock_start_of_week - demand[i], 0)
        stock.append(stock_end_of_week)
    cost.append(
        cost_per_storaged_unit * stock[i] + production_change_cost * int(weekly_production_rate[i] != weekly_production_rate[i - 1])
    )
# Create a DataFrame to store the results
df = pd.DataFrame({
    'Week': np.tile(np.arange(1, week_amount + 1), simulation_amount * len(u_values)),
    'Simulation': np.tile(np.repeat(np.arange(1, simulation_amount + 1), week_amount), len(u_values)),
    'U': upper_bound,
    'Stock': stock,
    'Production rate': weekly_production_rate,
    'Cost': cost,
    'Demand': demand
})
# Save the DataFrame to a CSV file
df.to_csv('simulation_results.csv', index=False)
df
    

,Week,Simulation,U,Stock,Production rate,Cost,Demand
0,1,1,30,60,120,1800,121
1,2,1,30,77,110,5310,93
2,3,1,30,49,110,1470,138
3,4,1,30,51,110,1530,108
4,5,1,30,44,110,1320,117
...,...,...,...,...,...,...,...
155995,48,500,80,38,110,1140,130
155996,49,500,80,30,110,900,118
155997,50,500,80,53,110,1590,87
155998,51,500,80,37,110,1110,126


## Obtención de costo total promedio
Tras realizar la suma de cada iteración y el promedio de todas estas, podemos apreciar que U = 50 es el valor que genera un menor costo.

In [ ]:
agg = df.groupby(['U', 'Simulation']).agg(
    Total_Cost=('Cost', 'sum'),
).groupby('U').agg(
    Average_Cost=('Total_Cost', 'mean'),
    Std_Cost=('Total_Cost', 'std'),
)
print('El mejor valor U =', agg.index[agg['Average_Cost'].argmin()])
agg

El mejor valor U = 50


,Average_Cost,Std_Cost
U,,
30,116949.78,11358.264061
40,109374.36,10618.958062
50,106756.08,10623.072341
60,107702.88,9938.864995
70,108988.56,10345.716274
80,112695.12,10655.227992


# Obtención de inventario final promedio

In [18]:
df.loc[df['Week'] == 52].groupby('U').agg(
    Average_Stock=('Stock', 'mean'),
    Std_Stock=('Stock', 'std'),
    Max_Stock=('Stock', 'max'),
    Min_Stock=('Stock', 'min'),
)

,Average_Stock,Std_Stock,Max_Stock,Min_Stock
U,,,,
30,33.122,19.677437,150,0
40,37.908,20.415904,115,0
50,42.022,21.045173,111,0
60,47.198,22.939993,122,0
70,50.070,25.182164,124,0
80,55.436,27.165084,131,0
